## Stack and Heap


_burton rosenberg, 28 june 2023_


### Table of contents.

1. <a href=#memoryrecap>Virtual memory recap</a>
1. <a href=#register>Register and Memory in the CPU</a>
1. <a href=#addrmodes>Addressing Modes</a>
1. <a href=#stackpointer>Stack Pointer</a>
1. <a href=#stackframe>The Stack Frame</a>
1. <a href=#stackpirates>Stack Pirates</a>
1. <a href=#heap>The Heap</a>


### <a name=memoryrecap>Virtual memory recap</a>

We have seen that we store data in virtual memory, which is an array of bytes from 0 to some highest index. It is valuable that the addresses are integers, because sequence of bytes can be collected by ascending indices to create, for example,

- multibyte integers, such as 4 bytes for a 32 bit integer'
- stack frames, whose size is the total needed storage for the local variables, and is determined by the compiler;
- the stack itself, as it grows down and returns up;
- strings, which are sequences of characters that extend indefinitely until a null byte `\0` is encountered;
- and arrays, where we jump to the i-th element by adding the address of the start of the array by a multiple of i, the multiple given by the `sizeof` the data element.


### <a name=register>Register and Memory in the CPU</a>



A very noticable difference in the use of memory between the hardware CPU and C Language is that C Language seems to operate on data items in the RAM. This is no the case in reality. The CPU can only do operations on data that is in its _registers_, memory locations on the CPU connected directly to the ALU. 


<div style="float:right;margin:2em;width:450px;border:1px solid green;padding:1em;">
<a title="MCS-80/85 Family User's Manual, Intel, October 1979" href="https://archive.org/details/Mcs80_85FamilyUsersManual/">
<img src="https://www.cs.miami.edu/home/burt/learning/csc421.241/images/mcs8085-addressing-modes.png">
<br>MCS-80/85 Family User's Manual, Intel, October 1979.
</a>
</div>

While the C programmer can write `k = i + j`, it compiler will turn this into machine code that,
- moves from RAM to register the values in `i` and `j`,
- instructs the ALU to add the contents of those two registers, leaving the result in some regiser,
- moves from the result register to the RAM `k` location the resulting value.

C languge very deliberately removed the concern of registers from the programmer. First, because it was tedious and did not contribute significantly to the quality of the software end product. Second, because the details of registers differ from chip to chip. Code that is written with concern for the registers can only be used on that or compatible chips.

### <a name=addrmodes>Addressing Modes</a>

The load and store from memory to register had a few ways of addressing the memory, which have certain analogies with C language data references. Without being specific on the particular chip architecture, here are some of the usual ways in which the memory location is specified,

- _Direct:_ The address is included in the instruction.
- _Register indirect:_ A register is named in the instruction. The value of that register is the memory address.
- _Register indexed:_ A register is named in the instruction, and other value is provided, either directly in the instruction or by naming a second register. The memory address indicated is the sum of the contents of these two.

These three sorts would be analogous in C to,

- _Direct:_ Load or store to the variable name.
- _Register indirect:_ Load or store from a pointer with the `*` notation.
- _Register indexed:_ Load or store using the array notation with an index selected. In the one case, the index is a constant, in the other case, the index is itself a variable.

### <a name=stackpointer>Stack Pointer</a>

Among the registers on a CPU one register is used exclusively for addressing, and has two additional features that enhance its addressing capabilites. This register is called the _stack pointer_.

The additional capabilites is that the register can increment itself after a load and decrement itself after a store. The store is called a _push_, and since the register decrements, repeated pushes will store the data in descending locations in the memory. The load is a called a _pop_, and since the register increments, repeated pops will load data from ascending locations in the memory.

In addition, a push or pop can occur where the data is the program counter. A _call_ will push the current program counter through the stack pointer, and load a new program counter value. A _return_ will pop into the program counter, thus resuming at the point in the code where occured the matching push.

That is what the hardware supplies as a mechanism. It is used by C to provide storage in an organized fashion for most of the variables in the program. 

All variables have, 

- a type,
- a scope,
- and a lifetime.

The stack provides the proper lifetime for variables by pushing them onto the stack at the beginning of a function call, and popping them off the stack at the end of the function call. And as well, the entry to the function is by a hardware call the records the address to return to onto this same stack; and the completion of the function uses the hardware return instruction to come back to that point.

It is easy to confuse _scope_ with _lifetime_. The _scope_ of a variable is the textual region of the code in which that variable name is known. It is completely the concern of the compiler. (And the program, who must understand the scope in the same way the compiler does.) The _lifetime_ of a variable is a runtime principle. It begins when space for the variable is allocated in the memory, and ends when the space is retired.

In the case of a recursive subroutine we can have a dramatic example of how scope and lifetime differ. If a recursive subroutine as a variable `i`, there is in a sense just one `i` and it has a scope as evident in reading the code, without concern for what the code does. 

Each time the recursive subroutine is called, new space is allocated for `i`, and again a different variable with the same name will be given new storage. Because of the stack, these instances will not be confused. The `i` in the current stack will be used for data and the others will be covered up, and their values will be uncovered later.

### <a name=stackframe>The Stack Frame</a>

In fact, the C compiler does not depend on push and pop to make room for new variables. The compile lays out a plan for all the variable in the function and prepares code that drops the stack on entry to the function by the total amount of space needed for this plan. The region enclosed by this drop is called the _stack frame_. 

The complier also prepares code that raises the stock on exit from the function. The _lifetime_ of variables on the stack begins at the stack drop and ends at the stack raising.

The function body then refers to all variables by stack pointer set to the bottom of the stack frame, plus a fixed offset that located the variable inside the stack frame.

It is a little different for the parameters to the function call. They are placed on the stack as well, but by the caller. When the call is made, the return address is pushed underneat the parameters, and we are now at the bottom of the caller's stack frame. At this point the function is entered, a new stack frame created, but the return address and parameters are easily located by an offset from the top of the new stack frame.

_Note:_ Top and bottom are ambiguous. I am here using top for higher numbered address and bottom for lower number addresses. And I am assuming that pushes decrement the stack pointer and pops increment the stack pointer.

### <a name=stackpirates>Stack Pirates</a>

We will do a pirate program to proof out the stack as it happens in a C langauge program. 

Pirating stacks is in fact a thing. One of the ways a hacker gets into your computer is by manipulating values on the stack in ways the cause the computer to run the hacker's program. We will set sail.


Here is what I propose we do. As the recursive function gets called inside itself, there is a local variable `j`. It will have space in its stack frame, a new space and a new stack frame for each recursive call to `recursive`. The code will note the location in memory of `j`, and store it away in an array (defined at the top level of the file, and hence in all the scope of all code in the file) called `stack_points`. We use the recursing variable `i` to guide this address into place. 

Once we get to the bottom of the recursion, the code prints out the list of addresses, and the contents at that address. At the point of `print_stack`, all the stack frames are in lifetime. Once begin to return back up the call stack the call frames go out of life time, but at moment of printing they are all in lifetime. 

This can vary by computer and implementation, but on this MacBook Pro M1, note that the spacing between locations is a regular 32 bytes (hex 0x20). 


In [18]:
%%file stack-pirates.c
#include<stdio.h>

#define N 5

int * stack_points[N] ;

void print_stack() {
    int i ;
    printf("stack address\tdata\n--------------------\n");
    for (i=0;i<N;i++) {
        printf("%p\t%d\n", *(stack_points+i), **(stack_points+i)) ;
    }
    return ;
}

int recursive(int i) {
    int j ; 
    if (i==N) {
        print_stack() ;
        return 0 ;
    }
    stack_points[i] = &j ;
    j = i ;
    recursive(i+1) ;
    return i ;
}

int main(int argc, char * argv[]){
    recursive(0) ;
    return 0 ;
}

Overwriting stack-pirates.c


In [22]:
%%bash
S=stack-pirates
cc -o $S $S.c
./$S
rm $S $S.c

stack address	data
--------------------
0x309de65a4	0
0x309de6584	1
0x309de6564	2
0x309de6544	3
0x309de6524	4


### <a name=heap>The Heap</a>

The compiler considers three classes of data,

- Local variables stored on the _stack_.
- Global and static variables stored on the _static area_.
- Dynamically allocated memory taking from a memory pool called the _heap_.

We discuss the heap, a pool of memory for dynamic allocation.

The compiler can create a string with the literal `hello, world!`. This would be static data, occupying storage in the static area. To dynamically create the string we would first need to allocate somewhere in memory a sufficiently long sequence of bytes.

To do this, the library call `malloc` is proved. It takes as an argument the number of bytes to allocate. It returns a pointer to the memory allocated. Because `malloc` does not know what type of data will be poured into this space, it returns a universal non-type, for `void *`, a pointer to void. You must cast the pointer to give the data in the space structure. For a string the structure is an array of char, so it is case with `(char *)`.

Given the space, the space can now be filled with data. And to retire the space, when not neeed, there is a `free` call. The free call is magical in that it takes only one argument, the pointer to the space to free. This must be a pointer returned by a `malloc`. The magic part is `free` knows how many bytes were allocated.

Rules:

- If you do not free, you might have a memory leak that will cause your program to run out of memory. However, all your memory will be returned when your program exists. The allocation is at longest for the lifetime of the program.
- Once you free a pointer, do not dererence it. The space might be reused and its contents completely inconsistent with what is expected. This causes bugs that are hard to fix, because the bug will show itself in what might be perfectly working code.
- Never dereference a null pointer. I bring this up because null pointers arrise in codes with malloc when the code path manages to neglect the malloc statement, under certain circumstances.


#### Note of memory layout

The stack and heap regions vary in size as the program runs. Typically a compiler were start the stack at the highest memory location available, and grows downward. The heap starts the lowest memory location and grows upwards. As long as the total size of heap and stack does not exceed the memory available, this is fine.

Below the heap region is typically the static variables and the code itself.

In [39]:
%%file my-strdup.c
#include<stdio.h>
#include<string.h>
#include<stdlib.h>

char * my_strdup(char * s) {
    char * r ;
    int i = 0 ;
    r = (char *) malloc(strlen(s)+1) ;
    while (s[i]) {
        r[i] = s[i] ;
        i++ ;
    }
    r[i] = '\0' ;
    return r ;
}

int main(int argc, char * argv[]) {
    char * s ;
    int i ;
    for (i=1; i<argc; i++) {
        s = my_strdup(argv[i]) ;
        printf("%s ", s) ; 
        free(s) ;
    }
    return 0 ;
}

Writing my-strdup.c


In [40]:
%%bash
S=my-strdup
cc -o $S $S.c
./$S hello world!
rm $S $S.c

hello world! 